In [4]:
import logging
import pdb
# logging.basicConfig(level=logging.INFO)
%matplotlib inline
%run LocalRepo.ipynb
%run repos.ipynb
%run parsing.ipynb
%run metrics.ipynb
%run analysis.ipynb
import pyfiglet
import numpy as np
# np.seterr(all='raise')

In [5]:
repos = [
    "eclipse/eclipse.jdt.core",  # from duerschmidt
    "ErikBrendel/LudumDare", # 8 minutes BTM
    "neuland/jade4j",
    "jfree/jfreechart",
    "brettwooldridge/HikariCP",  # 1.2h BTM
    "adamfisk/LittleProxy",  # 1.5h BTM
    "dynjs/dynjs",  # 2.5h BTM
    # "SonarSource/sonarqube",  # quite big / prob. several hours BTM
    "square/okhttp",
    "jenkinsci/jenkins",
    "eclipse/che",
    "elastic/elasticsearch",
    "apache/camel"
]

In [ ]:
views = ["structural", "evolutionary"] # ["structural", "evolutionary", "linguistic"]
for repo in repos:
    r = LocalRepo(repo)
    print(pyfiglet.figlet_format(r.name))
    r.update()
    
    # pdb.set_trace()
    print("Nodes in project structure:", r.get_tree().node_count())
    
    for view in views:
        # MetricManager.clear(r, view)

        coupling_graph = MetricManager.get(r, view)
        # coupling_graph.plaintext_save(repo)
        # coupling_graph.html_save(repo)
        

        print("\nResults from the " + view + " view:\n")

        coupling_graph.print_statistics()
        # coupling_graph.print_most_linked_nodes()
        # coupling_graph.show_weight_histogram()
        # coupling_graph.visualize(use_spring=False, with_labels=False)
        print("\n")
        
    #analyze_disagreements([MetricManager.get(r, view) for view in ["structural", "evolutionary", "linguistic"]], [
    #    [0, 1, None, "duplicated functionality that gets adjusted together"],
    #    [0, 0, 1, "duplicated concepts, developed separately"],
    #    [None, 0, 1, "duplicated concepts, adjusted separately"],
    #    [1, None, 0, "weakly modularized code"]
    #])
    analyze_disagreements([MetricManager.get(r, view) for view in ["structural", "evolutionary"]], [
        [0, 1, "duplicated functionality that gets adjusted together"],
    ])
    
print("\nProgram is over!")

           _ _                   __        _ _                   _     _ _     
  ___  ___| (_)_ __  ___  ___   / /__  ___| (_)_ __  ___  ___   (_) __| | |_   
 / _ \/ __| | | '_ \/ __|/ _ \ / / _ \/ __| | | '_ \/ __|/ _ \  | |/ _` | __|  
|  __/ (__| | | |_) \__ \  __// /  __/ (__| | | |_) \__ \  __/_ | | (_| | |_ _ 
 \___|\___|_|_| .__/|___/\___/_/ \___|\___|_|_| .__/|___/\___(_)/ |\__,_|\__(_)
              |_|                             |_|             |__/             
                    
  ___ ___  _ __ ___ 
 / __/ _ \| '__/ _ \
| (_| (_) | | |  __/
 \___\___/|_|  \___|
                    

updating eclipse/eclipse.jdt.core...
Repo is up to date!
Analyzing 8352 files...
Found 88425 classes, methods and fields!
Nodes in project structure: 88425
Using precalculated structural values

Results from the structural view:

WeightGraph statistics: 16578 nodes, 27838 edges, 142 connected component(s), with sizes: [15189, 45, 43, 40, 39, 39, 39, 33, 28, 27, 26, 22, 21, 20, 20, 17, 15, 1